<a href="https://colab.research.google.com/github/Armando5347/chatbot-imitador/blob/main/prueba_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PRUEBA DE USO DE UN LLM DE HUGGING FACE, EN CONCRETO, *openGPT-X*

Instalaciones e imports

In [1]:
!pip install transformers[torch]
!python -m pip install numpy torch huggingface_hub transformers sentencepiece datasets
import torch
import transformers
#from datasets import list_datasets
from datasets import DatasetDict, Dataset,load_dataset
import pandas as pd
import matplotlib.pyplot as plt
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import DataCollatorForLanguageModeling, TrainingArguments,Trainer

#from transformers import BertTokenizer, TrainingArguments, BertForSequenceClassification, Trainer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


Cargar el Modelo

In [2]:
#import torch
#from transformers import AutoModelForCausalLM, AutoTokenizer
device = torch.device(""" "cuda" if torch.cuda.is_available() else """ "cpu")
model_name = "openGPT-X/Teuken-7B-instruct-commercial-v0.4"
#generadro de texto del modelo openGPT-X
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
)
model = model.to(device).eval()
#tokenizer del modelo preentrenado
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast=False,
    trust_remote_code=True,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

...

A new version of the following files was downloaded from https://huggingface.co/openGPT-X/Teuken-7B-instruct-commercial-v0.4:
- gptx_tokenizer.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


special_tokens_map.json:   0%|          | 0.00/7.44k [00:00<?, ?B/s]

Found tokenizer model file: tokenizer.model


tokenizer.model:   0%|          | 0.00/4.72M [00:00<?, ?B/s]

Downloaded tokenizer model to: /root/.cache/huggingface/hub/models--openGPT-X--Teuken-7B-instruct-commercial-v0.4/snapshots/e91cc26f0a5b1082c1375fd161ccb8cc0a05b291/tokenizer.model


Prueba de entrenamiento con 'dataset' extra

In [3]:
#Un mini ejemplo, primero se tienen datasets de entrenamiento, evaluación y validación
#este de aquí debe de ser más grande, con muchos mensajes, (seguramente concatenados)
context_length = 8

archivo_csv = "./corpus_raw_emilio.csv"
mensajes = load_dataset("csv", data_files=archivo_csv, delimiter="\t")
mensajes = mensajes.rename_column("mensaje", "texto")

print(mensajes)

mensajes = mensajes["train"].train_test_split(test_size=0.2, seed=42)

mensajes["train"].save_to_disk("dataset_train")
mensajes["test"].save_to_disk("dataset_test")
# Datos
#mensajes_train = ["¡Hola gente!, soy Armando", "buenas pibes", "a que hora sale el COD?", "les caigo a las 9:00"]
#mensajes_eval = ["Hoy no creo poder conectarme", "Sale el proximo jueves un wings?", "a que hora sacan los pibes?"]

# Crear datasets
"""
datasets = DatasetDict(
    {
        "train": Dataset.from_dict({"texto": mensajes_train}),
        "eval": Dataset.from_dict({"texto": mensajes_eval}),
    }
)

print("Original datasets:", datasets)
"""
print(mensajes)
# Tokenización
def tokenizado(elemento):
  salida = tokenizer(
        elemento["texto"],
        padding=True,
        truncation=True,
        max_length=context_length,  # Limitar longitud máxima
        return_tensors=None,  # Retorna listas en lugar de tensores
        return_length=True
  )
  """
  input_batch = []
  for length, input_ids in zip(salida["length"], salida["input_ids"]):
    if length == context_length:
      input_batch.append(input_ids)
  print(input_batch)
  return {"input_ids": input_batch}
  """
  return {"input_ids": salida["input_ids"], "attention_mask": salida["attention_mask"]}

# Aplicar tokenización
datasets_tokenizados = mensajes.map(tokenizado, batched=True, remove_columns=mensajes["train"].column_names)

# Visualizar datasets tokenizados
print("Datasets tokenizados:", datasets_tokenizados)

#el data collator, a lo que entendí, este se encarga de separar en secciones o 'batches', aparte de crear las 'etiquetas', que son las mismas entradas, para esta tarea en particular
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

#hiperparametros para el entrenamiento
#este luego se pone feo
training_args = TrainingArguments(
    output_dir="prueba-entrenamiento",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="steps",
    eval_steps=1_000,
    logging_steps=1_000,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    weight_decay=0.1,
    warmup_steps=200,
    lr_scheduler_type="cosine",
    learning_rate=5e-3,
    save_steps=1_000,
    fp16=True,
    report_to="none"  #me parece que esta linea es para que  no pida lig-in en weights and biases
)

#este se encarga de entrenar con el modelo, los parametros, un conjunto de datos de prueba y validación
trainer = Trainer(
    model,
    training_args,
    train_dataset=datasets_tokenizados['train'],
    eval_dataset=datasets_tokenizados['test'],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()



Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['texto'],
        num_rows: 3110
    })
})


Saving the dataset (0/1 shards):   0%|          | 0/2488 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/622 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['texto'],
        num_rows: 2488
    })
    test: Dataset({
        features: ['texto'],
        num_rows: 622
    })
})


Map:   0%|          | 0/2488 [00:00<?, ? examples/s]

Map:   0%|          | 0/622 [00:00<?, ? examples/s]

Datasets tokenizados: DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 2488
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 622
    })
})


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-3-59ff9bd2410a>:81: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


KeyboardInterrupt: 

Prueba del modelo

In [ ]:
messages = [{"role": "User", "content": "Hola!, ¿cómo estas?"}]
prompt_ids = tokenizer.apply_chat_template(messages, chat_template="ES", tokenize=True, add_generation_prompt=True, return_tensors="pt")
prediction = model.generate(
    prompt_ids.to(model.device),
    max_length=512,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.7,
    num_return_sequences=1,
)
prediction_text = tokenizer.decode(prediction[0].tolist())
print(prediction_text)
